#사전 작업

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!pip install shap
!pip install catboost


#데이터 전처리


In [ ]:
# 필요한 패키지들 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import re
import shap
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error


# 폰트 설정
plt.rc('font', family='NanumBarunGothic')

# 데이터 불러오기
file_path = "/content/drive/MyDrive/Colab Notebooks/01 DATA/accidentInfoList.CSV"
df = pd.read_csv(file_path, encoding='cp949')

In [ ]:
# 숫자 추출 함수
def extract_numbers_from_age(age):
    numbers = re.findall(r'\d+', str(age))
    if numbers:
        return int(numbers[0])
    else:
        return np.nan

# '가해운전자 연령' 및 '피해운전자 연령'에서 숫자만 추출
df['가해운전자 연령'] = df['가해운전자 연령'].apply(extract_numbers_from_age)
df['피해운전자 연령'] = df['피해운전자 연령'].apply(extract_numbers_from_age)

# 결측치 처리
mean_age_driver = round(df['가해운전자 연령'].mean(), 1)
mean_age_victim = round(df['피해운전자 연령'].mean(), 1)
df['가해운전자 연령'].fillna(mean_age_driver, inplace=True)
df['피해운전자 연령'].fillna(mean_age_victim, inplace=True)
df['피해운전자 차종'].fillna('기타불명', inplace=True)
df['피해운전자 성별'].fillna('기타불명', inplace=True)

In [ ]:
# 피처 엔지니어링
# 'TAD' 피처 생성
df['TAD'] = df['사망자수'] * 10 + df['중상자수'] * 5 + df['경상자수'] * 3 + df['부상신고자수'] * 1
df.drop(columns=['사망자수', '경상자수', '중상자수', '부상신고자수'], inplace=True)

# '사고일시'에서 시간 정보 추출
df['사고일시'] = pd.to_datetime(df['사고일시'], format='%Y년 %m월 %d일 %H시')
df['사고발생시간'] = df['사고일시'].dt.hour
df.drop(columns=['사고일시'], inplace=True)

# 피처 제거
df.drop(columns=['사고번호', '시군구','사고내용'], inplace=True)

In [ ]:
#범주형 데이터 처리
# 요일 범주화
def categorize_weekday(weekday):
    if weekday in ['월요일', '화요일', '수요일', '목요일', '금요일']:
        return '평일'
    else:
        return '주말'

df['평일/주말'] = df['요일'].apply(categorize_weekday)
df.drop(columns=['요일'], inplace=True)

# 사고발생시간 범주화
def recategorize_time_of_day(hour):
    if 21 <= hour <= 24 or 0 <= hour < 6:
        return '야간'
    else:
        return '주간'

df['시간대'] = df['사고발생시간'].apply(recategorize_time_of_day)
df.drop(columns=['사고발생시간'], inplace=True)

# 연령대 범주화
def categorize_age(age):
    if age <= 19:
        return '미성년층'
    elif 20 <= age <= 45:
        return '청년층'
    elif 46 <= age <= 60:
        return '중장년층'
    else:
        return '노년층'

df['가해운전자 연령대'] = df['가해운전자 연령'].apply(categorize_age)
df.drop(columns=['가해운전자 연령'], inplace=True)
df['피해운전자 연령대'] = df['피해운전자 연령'].apply(categorize_age)
df.drop(columns=['피해운전자 연령'], inplace=True)

# 노면상태 범주화
dangerous_conditions = ['서리/결빙', '적설', '침수', '해빙', '기타']
df['노면상태_위험도'] = df['노면상태'].apply(lambda x: '위험(적설, 침수)' if x in dangerous_conditions else x)
df.drop(columns=['노면상태'], inplace=True)

#기상상태 범주화
df['기상상태_위험도'] = df['기상상태'].apply(lambda x: '매우 위험(안개, 눈)' if x in ['안개', '눈'] else ('위험(비, 흐림)' if x in ['비', '흐림'] else x))
df.drop(columns=['기상상태'], inplace=True)

# 가해/피해운전자 차종 범주화
large_vehicle_types = ['건설기계', '승용', '승합', '특수', '화물', '농기계']
small_vehicle_types = ['개인형이동수단(PM)', '기타불명', '사륜오토바이(ATV)', '원동기', '이륜', '자전거']

def classify_driver_vehicle_type(vehicle_type):
    if vehicle_type in large_vehicle_types:
        return '중대형'
    elif vehicle_type in small_vehicle_types:
        return '소형'
    else:
        return vehicle_type

df['가해운전자 차종_분류'] = df['가해운전자 차종'].apply(classify_driver_vehicle_type)
df['피해운전자 차종_분류'] = df['피해운전자 차종'].apply(classify_driver_vehicle_type)
df.drop(columns=['가해운전자 차종', '피해운전자 차종'], inplace=True)

# 도로형태 명칭 정리
def categorize_road_type(road_type):
    if road_type == '단일로 - 기타':
        return '단일로 - 일반'
    else:
        return road_type

df['도로형태'] = df['도로형태'].apply(categorize_road_type)



In [ ]:
# 인코딩
categorical_features = ['사고유형', '법규위반', '노면상태_위험도', '기상상태_위험도', '도로형태',
                        '가해운전자 차종_분류', '가해운전자 성별', '피해운전자 차종_분류',
                        '피해운전자 성별', '평일/주말', '시간대',
                        '가해운전자 연령대', '피해운전자 연령대','가해운전자 상해정도', '피해운전자 상해정도']

df = pd.get_dummies(df, columns=categorical_features)


In [ ]:
# 이상치 제거
z_scores = np.abs(stats.zscore(df['TAD']))
outliers = (z_scores > 3)
df_no_outliers = df[~outliers]

# MINMAX 스케일러 적용

minmax_scaler = MinMaxScaler()
TAD_minmax_scaled = minmax_scaler.fit_transform(df_no_outliers['TAD'].values.reshape(-1, 1))
df_no_outliers['TAD_scaled'] = TAD_minmax_scaled



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#모델링

In [ ]:
# 피처 선택
features = ['사고유형_차대사람 - 길가장자리구역통행중', '사고유형_차대사람 - 보도통행중', '사고유형_차대사람 - 차도통행중', '사고유형_차대사람 - 횡단중', '사고유형_차대차 - 정면충돌', '사고유형_차대차 - 추돌', '사고유형_차대차 - 측면충돌', '사고유형_차대차 - 후진중충돌', '사고유형_차량단독 - 공작물충돌', '사고유형_차량단독 - 도로외이탈 - 추락', '사고유형_차량단독 - 전도전복 - 전도', '사고유형_차량단독 - 전도전복 - 전복', '사고유형_차량단독 - 주/정차차량 충돌', '법규위반_과속', '법규위반_교차로운행방법위반', '법규위반_보행자보호의무위반', '법규위반_불법유턴', '법규위반_신호위반', '법규위반_안전거리미확보', '법규위반_안전운전불이행', '법규위반_중앙선침범', '법규위반_직진우회전진행방해', '법규위반_차로위반', '노면상태_위험도_건조', '노면상태_위험도_위험(적설, 침수)', '노면상태_위험도_젖음/습기', '기상상태_위험도_맑음', '기상상태_위험도_매우 위험(안개, 눈)', '기상상태_위험도_위험(비, 흐림)', '도로형태_교차로 - 교차로부근', '도로형태_교차로 - 교차로안', '도로형태_교차로 - 교차로횡단보도내', '도로형태_단일로 - 고가도로위', '도로형태_단일로 - 교량','도로형태_단일로 - 일반', '도로형태_단일로 - 지하차도(도로)내', '도로형태_단일로 - 터널', '도로형태_미분류 - 미분류', '도로형태_주차장 - 주차장', '가해운전자 차종_분류_소형', '가해운전자 차종_분류_중대형', '가해운전자 성별_남', '가해운전자 성별_여', '피해운전자 차종_분류_보행자', '피해운전자 차종_분류_소형', '피해운전자 차종_분류_중대형', '피해운전자 성별_남', '피해운전자 성별_여', '평일/주말_주말', '평일/주말_평일', '시간대_야간', '시간대_주간', '가해운전자 연령대_노년층', '가해운전자 연령대_미성년층', '가해운전자 연령대_중장년층', '가해운전자 연령대_청년층', '피해운전자 연령대_노년층', '피해운전자 연령대_미성년층', '피해운전자 연령대_중장년층', '피해운전자 연령대_청년층', '가해운전자 상해정도_경상', '가해운전자 상해정도_부상신고', '가해운전자 상해정도_사망', '가해운전자 상해정도_상해없음', '가해운전자 상해정도_중상', '피해운전자 상해정도_경상', '피해운전자 상해정도_부상신고', '피해운전자 상해정도_사망', '피해운전자 상해정도_상해없음', '피해운전자 상해정도_중상']


# 훈련 데이터 및 테스트 데이터 분할
X = df_no_outliers[features]
y = df_no_outliers['TAD_scaled']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

xgb_tuned = XGBRegressor(subsample=0.7, n_estimators=300, min_child_weight=1, max_depth=10, learning_rate=0.05, gamma=0.1, colsample_bytree=0.8, random_state=42)
xgb_tuned.fit(X_train, y_train)

# CatBoost 모델 훈련
catboost_tuned = CatBoostRegressor(learning_rate=0.05, l2_leaf_reg=9, depth=4, random_seed=42, verbose=0)
catboost_tuned.fit(X_train, y_train)

def evaluate_model_performance(model_name, model, X_train, y_train, X_test, y_test):
    # 훈련 세트 예측
    y_pred_train = model.predict(X_train)
    # 테스트 세트 예측
    y_pred_test = model.predict(X_test)

    print(f"\n{model_name} 모델 성능 평가:")
    print("훈련 세트 R2 점수:", r2_score(y_train, y_pred_train))
    print("테스트 세트 R2 점수:", r2_score(y_test, y_pred_test))
    print("훈련 세트 MSE:", mean_squared_error(y_train, y_pred_train))
    print("테스트 세트 MSE:", mean_squared_error(y_test, y_pred_test))
    print("훈련 세트 MAE:", mean_absolute_error(y_train, y_pred_train))
    print("테스트 세트 MAE:", mean_absolute_error(y_test, y_pred_test))


# XGBoost와 CatBoost 모델을 앙상블
ensemble_model = VotingRegressor([('XGBoost', xgb_tuned), ('CatBoost', catboost_tuned)])
ensemble_model.fit(X_train, y_train)

# 앙상블 모델 평가
evaluate_model_performance("Ensemble (XGBoost + CatBoost)", ensemble_model, X_train, y_train, X_test, y_test)




Ensemble (XGBoost + CatBoost) 모델 성능 평가:
훈련 세트 R2 점수: 0.4748770824900067
테스트 세트 R2 점수: 0.460338721038575
훈련 세트 MSE: 0.016658922797898167
테스트 세트 MSE: 0.016963739429337172
훈련 세트 MAE: 0.07844653273923775
테스트 세트 MAE: 0.07945072856007061


In [ ]:
import shap

# 각 개별 모델에 대해 SHAP 값 계산
explainer_xgb = shap.Explainer(xgb_tuned)
shap_values_xgb = explainer_xgb.shap_values(X_train)

explainer_catboost = shap.Explainer(catboost_tuned)
shap_values_catboost = explainer_catboost.shap_values(X_train)

# 앙상블
shap_values_ensemble = (shap_values_xgb + shap_values_catboost) / 2

print("SHAP values for each feature:")
for i, feature in enumerate(X_train.columns):
    print(f"{feature}: {shap_values_ensemble[:, i].mean() * 10000}")

SHAP values for each feature:
사고유형_차대사람 - 길가장자리구역통행중: 0.0009907895170435572
사고유형_차대사람 - 보도통행중: -4.1337498469787955e-05
사고유형_차대사람 - 차도통행중: 0.02229931031864355
사고유형_차대사람 - 횡단중: 0.10010815596932848
사고유형_차대차 - 정면충돌: -0.08119327715693553
사고유형_차대차 - 추돌: -1.765063915037323
사고유형_차대차 - 측면충돌: 0.5351874150648019
사고유형_차대차 - 후진중충돌: 0.36016617888005853
사고유형_차량단독 - 공작물충돌: -0.2751702190380444
사고유형_차량단독 - 도로외이탈 - 추락: 0.00039872607131541415
사고유형_차량단독 - 전도전복 - 전도: -0.006391187039953154
사고유형_차량단독 - 전도전복 - 전복: -0.014042528774484957
사고유형_차량단독 - 주/정차차량 충돌: 0.0
법규위반_과속: 0.027375067214972345
법규위반_교차로운행방법위반: -0.12492352673632633
법규위반_보행자보호의무위반: 1.5775710199293025
법규위반_불법유턴: 0.022769483078173693
법규위반_신호위반: 0.68020862884138
법규위반_안전거리미확보: 0.4058230589983506
법규위반_안전운전불이행: -0.15786131181493673
법규위반_중앙선침범: 0.021858940353576963
법규위반_직진우회전진행방해: 0.01251670018637398
법규위반_차로위반: -0.008465181634163776
노면상태_위험도_건조: 0.05530486915012396
노면상태_위험도_위험(적설, 침수): 0.033716281953446295
노면상태_위험도_젖음/습기: 0.0328636561451724
기상상태_위험도_맑음: 0

#시각화 관련

In [ ]:
# 빈도표
plt.figure(figsize=(16, 12))
sns.countplot(x='피해운전자 연령대', data=df, palette='viridis')
plt.ylabel('빈도')
plt.show()


In [ ]:
# 커널 밀도 추정 그래프
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x='사고발생시간', fill=True, color='skyblue')
plt.title('사고발생시간별 분포')
plt.xlabel('사고발생시간')
plt.ylabel('밀도')
plt.show()


In [ ]:
# 사고내용별 ECLO
avg_eclo_by_accident = df.groupby('시군구')['TAD'].mean().reset_index()
avg_eclo_all = df['TAD'].mean()

# 막대 그래프
plt.figure(figsize=(12, 8))
sns.barplot(x='시군구', y='TAD', data=avg_eclo_by_accident, palette='viridis')
plt.ylabel('평균 ETAD')
plt.axhline(avg_eclo_all, color='r', linestyle='--', label='전체 평균 TAD')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def categorize_age(age):
    if age <= 19:
        return '0~19'
    elif age <= 45:
        return '20~45'
    elif age <= 60:
        return '46~60'
    else:
        return '61 이상'

# 연령을 범주
df['피해운전자 연령대'] = df['피해운전자 연령'].apply(categorize_age)

# 사고내용별 ECLO
avg_eclo_by_accident = df.groupby('피해운전자 연령대')['TAD'].mean().reset_index()
avg_eclo_all = df['ECLO'].mean()

# 막대 그래프
plt.figure(figsize=(12, 8))
sns.barplot(x='피해운전자 연령대', y='TAD', data=avg_eclo_by_accident, palette='viridis')
plt.ylabel('평균 TAD')
plt.axhline(avg_eclo_all, color='r', linestyle='--', label='전체 평균 TAD')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3가지 이상의 피처를 활용한 히트맵

heatmap_data = df.pivot_table(index='사고발생시간', columns='법규위반', values='TAD', aggfunc='mean')

plt.figure(figsize=(14, 10))
sns.heatmap(heatmap_data, cmap='viridis', annot=True, fmt=".2f", linewidths=.5)
plt.title('사고 발생시간과 법규위반에 따른 TAD 평균')
plt.xlabel('법규위반')
plt.ylabel('사고발생시간')
plt.show()
